In [29]:
from jax import jit, random
import pandas as pd
from datetime import datetime, timedelta
import numpyro
from numpyro import distributions as dist
from numpyro import infer
import arviz as az
from IPython.display import Markdown
from plotly.express.colors import qualitative as qual_colours
from pathlib import Path
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Markdown

from estival.sampling import tools as esamp

from emu_renewal.process import CosineMultiCurve
from emu_renewal.distributions import GammaDens
from emu_renewal.renew import RenewalDeathsModel, ModelDeathsResult
from emu_renewal.outputs import get_spaghetti, get_quant_df_from_spaghetti
from emu_renewal.plotting import plot_uncertainty_patches, plot_3d_spaghetti, plot_post_prior_comparison, plot_spaghetti_calib_comparison
from emu_renewal.calibration import StandardCalib
from emu_renewal.utils import get_adjust_idata_index, adjust_summary_cols
from emu_renewal.targets import StandardTarget

In [30]:
PROJECT_PATH = Path.cwd().resolve()
DATA_PATH = PROJECT_PATH.parent / "data/covid_aus"

In [31]:
# Get data
target_data = pd.read_csv(DATA_PATH / "WHO-COVID-19-global-data_21_8_24.csv")
seroprev_data = pd.read_csv(DATA_PATH / "aus_seroprev_data.csv")

# Clean cases data
aust_data = target_data.loc[target_data["Country"] == "Australia"]
aust_data.index = pd.to_datetime(aust_data["Date_reported"], format="%d/%m/%Y")
aust_cases = aust_data["New_cases"].resample("W-SUN").interpolate(method="linear").fillna(0.0)
aust_deaths = aust_data["New_deaths"]

# Clean seroprevalence data
seroprev_data.index = pd.to_datetime(seroprev_data["date"])
aust_seroprev = seroprev_data["seroprevalence"]

# Hospitalisation data
aust_hosp = pd.read_pickle(DATA_PATH / "hosp.pkl")

In [ ]:
# Specify fixed parameters and get calibration data
proc_update_freq = 14
init_time = 50
pop = 26e6
analysis_start = datetime(2021, 12, 1)
analysis_end = datetime(2022, 10, 1)
init_start = analysis_start - timedelta(init_time)
init_end = analysis_start - timedelta(1)
select_data = aust_cases.loc[analysis_start: analysis_end]
select_deaths = aust_deaths.loc[analysis_start: analysis_end]
hosp_data = aust_hosp[analysis_start: analysis_end]
init_data = aust_cases.resample("D").asfreq().interpolate().loc[init_start: init_end] / 7.0

In [ ]:
# Define model and fitter
proc_fitter = CosineMultiCurve()
renew_model = RenewalDeathsModel(pop, analysis_start, analysis_end, proc_update_freq, proc_fitter, GammaDens(), init_time, init_data, GammaDens())

In [ ]:
# Define parameter ranges
priors = {
    "gen_mean": dist.TruncatedNormal(7.3, 0.5, low=1.0),
    "gen_sd": dist.TruncatedNormal(3.8, 0.5, low=1.0),
    "cdr": dist.Beta(15, 15), #(16,40)
    "ifr": dist.Beta(3, 20),
    "rt_init": dist.Normal(0.0, 0.25),
    "report_mean": dist.TruncatedNormal(8, 0.5, low=1.0),
    "report_sd": dist.TruncatedNormal(3, 0.5, low=1.0),
    "death_mean": dist.TruncatedNormal(18.0, 0.5, low=1.0),
    "death_sd": dist.TruncatedNormal(5.0, 0.5, low=1.0),
}

In [ ]:
# Define model and fitter
proc_fitter = CosineMultiCurve()
renew_model = RenewalDeathsModel(pop, analysis_start, analysis_end, proc_update_freq, proc_fitter, GammaDens(), init_time, init_data, GammaDens())

In [ ]:
# Define calibration and calib data
calib_data = {
    "weekly_sum": StandardTarget(select_data, 0.1),
    "seropos": StandardTarget(aust_seroprev, 0.1),
    "weekly_deaths": StandardTarget(select_deaths, 0.1),
}
calib = StandardCalib(renew_model, priors, calib_data)

In [ ]:
# Run calibration
kernel = infer.NUTS(calib.calibration, dense_mass=True, init_strategy=calib.custom_init(radius=0.5))
mcmc = infer.MCMC(kernel, num_chains=2, num_samples=100, num_warmup=100)
mcmc.run(random.PRNGKey(1))

In [ ]:
# Grab sample of data from calibrated model outputs
idata = az.from_dict(mcmc.get_samples(True))
idata_sampled = az.extract(idata, num_samples=20)
sample_params = esamp.xarray_to_sampleiterator(idata_sampled)

In [ ]:
spaghetti = get_spaghetti(calib, sample_params)

In [ ]:
plot_spaghetti_calib_comparison(spaghetti, calib.targets, ["weekly_sum", "weekly_deaths", "seropos"])

In [ ]:
az.summary(idata)

In [ ]:
plot_post_prior_comparison(idata, list(priors.keys()), priors);